# Implementácia expertného systému pre zvieratá

Príklad z [AI for Beginners Curriculum](http://github.com/microsoft/ai-for-beginners).

V tomto príklade implementujeme jednoduchý vedomosťami riadený systém na určenie zvieraťa na základe niektorých fyzických charakteristík. Systém možno znázorniť nasledujúcim stromom AND-OR (toto je časť celého stromu, môžeme ľahko pridať ďalšie pravidlá):

![](../../../../../../translated_images/sk/AND-OR-Tree.5592d2c70187f283.webp)


## Náš vlastný shell expertných systémov s spätovým odvodzovaním

Skúsme definovať jednoduchý jazyk pre reprezentáciu poznatkov založený na produkčných pravidlách. Ako kľúčové slová na definovanie pravidiel použijeme triedy v Pythone. V podstate by existovali 3 typy tried:
* `Ask` predstavuje otázku, ktorá sa musí položiť používateľovi. Obsahuje množinu možných odpovedí.
* `If` predstavuje pravidlo a je to len syntaktický cukor na uloženie obsahu pravidla
* `AND`/`OR` sú triedy na reprezentáciu AND/OR vetiev stromu. Len ukladajú zoznam argumentov vo vnútri. Na zjednodušenie kódu je všetka funkcionalita definovaná v nadradenej triede `Content`


In [1]:
class Ask():
    def __init__(self,choices=['y','n']):
        self.choices = choices
    def ask(self):
        if max([len(x) for x in self.choices])>1:
            for i,x in enumerate(self.choices):
                print("{0}. {1}".format(i,x),flush=True)
            x = int(input())
            return self.choices[x]
        else:
            print("/".join(self.choices),flush=True)
            return input()

class Content():
    def __init__(self,x):
        self.x=x
        
class If(Content):
    pass

class AND(Content):
    pass

class OR(Content):
    pass

V našom systéme by pracovná pamäť obsahovala zoznam **faktov** ako **párov atribút-hodnota**. Základňa znalostí môže byť definovaná ako jeden veľký slovník, ktorý mapuje akcie (nové fakty, ktoré by mali byť vložené do pracovnej pamäti) na podmienky, vyjadrené ako AND-OR výrazy. Tiež niektoré fakty môžu byť `Ask`-nuté.


In [2]:
rules = {
    'default': Ask(['y','n']),
    'color' : Ask(['red-brown','black and white','other']),
    'pattern' : Ask(['dark stripes','dark spots']),
    'mammal': If(OR(['hair','gives milk'])),
    'carnivor': If(OR([AND(['sharp teeth','claws','forward-looking eyes']),'eats meat'])),
    'ungulate': If(['mammal',OR(['has hooves','chews cud'])]),
    'bird': If(OR(['feathers',AND(['flies','lies eggs'])])),
    'animal:monkey' : If(['mammal','carnivor','color:red-brown','pattern:dark spots']),
    'animal:tiger' : If(['mammal','carnivor','color:red-brown','pattern:dark stripes']),
    'animal:giraffe' : If(['ungulate','long neck','long legs','pattern:dark spots']),
    'animal:zebra' : If(['ungulate','pattern:dark stripes']),
    'animal:ostrich' : If(['bird','long nech','color:black and white','cannot fly']),
    'animal:pinguin' : If(['bird','swims','color:black and white','cannot fly']),
    'animal:albatross' : If(['bird','flies well'])
}

Na vykonanie spätného odvodzovania definujeme triedu `Knowledgebase`. Bude obsahovať:
* Pracovnú `memory` - slovník, ktorý mapuje atribúty na hodnoty
* Pravidlá Knowledgebase `rules` v hore uvedenom formáte

Dve hlavné metódy sú:
* `get` na získanie hodnoty atribútu, pričom v prípade potreby vykoná odvodzovanie. Napríklad `get('color')` získa hodnotu farby (v prípade potreby sa spýta a uloží hodnotu na neskoršie použitie v pracovnej pamäti). Ak sa spýtame `get('color:blue')`, opýta sa na farbu a potom vráti hodnotu `y`/`n` podľa farby.
* `eval` vykonáva samotné odvodzovanie, t.j. prechádza stromom AND/OR, vyhodnocuje podciele atď.


In [3]:
class KnowledgeBase():
    def __init__(self,rules):
        self.rules = rules
        self.memory = {}
        
    def get(self,name):
        if ':' in name:
            k,v = name.split(':')
            vv = self.get(k)
            return 'y' if v==vv else 'n'
        if name in self.memory.keys():
            return self.memory[name]
        for fld in self.rules.keys():
            if fld==name or fld.startswith(name+":"):
                # print(" + proving {}".format(fld))
                value = 'y' if fld==name else fld.split(':')[1]
                res = self.eval(self.rules[fld],field=name)
                if res!='y' and res!='n' and value=='y':
                    self.memory[name] = res
                    return res
                if res=='y':
                    self.memory[name] = value
                    return value
        # field is not found, using default
        res = self.eval(self.rules['default'],field=name)
        self.memory[name]=res
        return res
                
    def eval(self,expr,field=None):
        # print(" + eval {}".format(expr))
        if isinstance(expr,Ask):
            print(field)
            return expr.ask()
        elif isinstance(expr,If):
            return self.eval(expr.x)
        elif isinstance(expr,AND) or isinstance(expr,list):
            expr = expr.x if isinstance(expr,AND) else expr
            for x in expr:
                if self.eval(x)=='n':
                    return 'n'
            return 'y'
        elif isinstance(expr,OR):
            for x in expr.x:
                if self.eval(x)=='y':
                    return 'y'
            return 'n'
        elif isinstance(expr,str):
            return self.get(expr)
        else:
            print("Unknown expr: {}".format(expr))

Teraz definujme našu databázu zvierat a vykonajme konzultáciu. Upozorňujem, že tento hovor vám bude klásť otázky. Môžete odpovedať zadaním `y`/`n` pre otázky áno-nie alebo zadaním čísla (0..N) pre otázky s viacnásobným výberom odpovede.


In [4]:
kb = KnowledgeBase(rules)
kb.get('animal')

hair
y/n
sharp teeth
y/n
claws
y/n
forward-looking eyes
y/n
color
0. red-brown
1. black and white
2. other
has hooves
y/n
long neck
y/n
long legs
y/n
pattern
0. dark stripes
1. dark spots


'giraffe'

## Použitie Experta na dopredné usudzovanie

V nasledujúcom príklade sa pokúsime implementovať dopredné usudzovanie pomocou jednej z knižníc pre reprezentáciu vedomostí, [Experta](https://github.com/nilp0inter/experta). **Experta** je knižnica na vytváranie systémov dopredného usudzovania v Pythone, ktorá je navrhnutá tak, aby bola podobná klasickému starému systému [CLIPS](http://www.clipsrules.net/index.html).

Dopredné reťazenie by sme mohli implementovať aj sami bez väčších problémov, ale naivné implementácie obyčajne nie sú veľmi efektívne. Pre efektívnejšie zodpovedanie pravidiel sa používa špeciálny algoritmus [Rete](https://en.wikipedia.org/wiki/Rete_algorithm).


In [5]:
import sys
!{sys.executable} -m pip install git+https://github.com/nilp0inter/experta

  Cloning https://github.com/nilp0inter/experta to /tmp/pip-req-build-7qurtwk3
  Running command git clone --filter=blob:none --quiet https://github.com/nilp0inter/experta /tmp/pip-req-build-7qurtwk3
  Resolved https://github.com/nilp0inter/experta to commit c6d5834b123861f5ae09e7d07027dc98bec58741
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for experta: filename=experta-1.9.5.dev1-py3-none-any.whl size=34804 sha256=888c459512a5e713f4b674caa9a0f96cfdf07ec0d6eb56cc318ce0653d218014
  Stored in directory: /tmp/pip-ephem-wheel-cache-1eeii9zy/wheels/3d/e8/bb/22d7956359603fa8dd679aa09f5b8efb3f29991c3986fdc787
Successfully built experta
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [experta]


In [13]:
from experta import *
#import experta

Náš systém definujeme ako triedu, ktorá je podtriedou `KnowledgeEngine`. Každé pravidlo je definované samostatnou funkciou s anotáciou `@Rule`, ktorá určuje, kedy by sa pravidlo malo spustiť. Vo vnútri pravidla môžeme pridať nové fakty pomocou funkcie `declare`, a pridanie týchto faktov spôsobí, že viac pravidiel bude vyvolaných dopredným inferenčným mechanizmom.


In [14]:
class Animals(KnowledgeEngine):
    @Rule(OR(
           AND(Fact('sharp teeth'),Fact('claws'),Fact('forward looking eyes')),
           Fact('eats meat')))
    def cornivor(self):
        self.declare(Fact('carnivor'))
        
    @Rule(OR(Fact('hair'),Fact('gives milk')))
    def mammal(self):
        self.declare(Fact('mammal'))

    @Rule(Fact('mammal'),
          OR(Fact('has hooves'),Fact('chews cud')))
    def hooves(self):
        self.declare('ungulate')
        
    @Rule(OR(Fact('feathers'),AND(Fact('flies'),Fact('lays eggs'))))
    def bird(self):
        self.declare('bird')
        
    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark spots'))
    def monkey(self):
        self.declare(Fact(animal='monkey'))

    @Rule(Fact('mammal'),Fact('carnivor'),
          Fact(color='red-brown'),
          Fact(pattern='dark stripes'))
    def tiger(self):
        self.declare(Fact(animal='tiger'))

    @Rule(Fact('ungulate'),
          Fact('long neck'),
          Fact('long legs'),
          Fact(pattern='dark spots'))
    def giraffe(self):
        self.declare(Fact(animal='giraffe'))

    @Rule(Fact('ungulate'),
          Fact(pattern='dark stripes'))
    def zebra(self):
        self.declare(Fact(animal='zebra'))

    @Rule(Fact('bird'),
          Fact('long neck'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def straus(self):
        self.declare(Fact(animal='ostrich'))

    @Rule(Fact('bird'),
          Fact('swims'),
          Fact('cannot fly'),
          Fact(color='black and white'))
    def pinguin(self):
        self.declare(Fact(animal='pinguin'))

    @Rule(Fact('bird'),
          Fact('flies well'))
    def albatros(self):
        self.declare(Fact(animal='albatross'))
        
    @Rule(Fact(animal=MATCH.a))
    def print_result(self,a):
          print('Animal is {}'.format(a))
                    
    def factz(self,l):
        for x in l:
            self.declare(x)

Akonáhle definujeme vedomostnú databázu, naplníme našu pracovnú pamäť niektorými počiatočnými faktami a potom zavoláme metódu `run()`, aby sme vykonali inferenciu. Výsledkom môžete vidieť, že do pracovnej pamäti sú pridané nové odvodené fakty, vrátane finálneho faktu o zvierati (ak sme správne nastavili všetky počiatočné fakty).


In [15]:
ex1 = Animals()
ex1.reset()
ex1.factz([
    Fact(color='red-brown'),
    Fact(pattern='dark stripes'),
    Fact('sharp teeth'),
    Fact('claws'),
    Fact('forward looking eyes'),
    Fact('gives milk')])
ex1.run()
ex1.facts

Animal is tiger


FactList([(0, InitialFact()),
          (1, Fact(color='red-brown')),
          (2, Fact(pattern='dark stripes')),
          (3, Fact('sharp teeth')),
          (4, Fact('claws')),
          (5, Fact('forward looking eyes')),
          (6, Fact('gives milk')),
          (7, Fact('mammal')),
          (8, Fact('carnivor')),
          (9, Fact(animal='tiger'))])

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vylúčenie zodpovednosti**:
Tento dokument bol preložený pomocou AI prekladateľskej služby [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa usilujeme o presnosť, vezmite prosím na vedomie, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nie sme zodpovední za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
